In [1]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
df = pd.read_csv('processed_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356251 entries, 0 to 356250
Columns: 248 entries, index to paymentrate
dtypes: bool(133), float64(71), int64(44)
memory usage: 357.8 MB


In [5]:
import mlflow.pyfunc

# Chemin vers le modèle dans MLflow, utilisant la run ID
model_uri = f"runs:/bd401b5ef2ad4eda893507c540f264f5/model.pkl"

# Charger le modèle
model = mlflow.pyfunc.load_model(model_uri)

# Tester le modèle avec des données
import pandas as pd

# Exemple de données
input_data = pd.DataFrame({
    'paymentrate': [100],
    'extsource3': [200],
    # Ajoute toutes les features utilisées
})

# Faire une prédiction
prediction = model.predict(input_data)
print(prediction)


OSError: No such file or directory: 'G:\projet data\OPC_Projets\P7\mlruns\420642211746036919\bd401b5ef2ad4eda893507c540f264f5\artifacts\model.pkl'

In [1]:
import joblib

# Chemin vers le modèle sauvegardé
model_path = 'G:/projet data/OPC_Projets/P7/mlruns/276709811813126569/73b32bb40f2d46f7a98bdade812a126b/artifacts/model/model.pkl'

# Charger le modèle depuis le fichier .pkl
best_lgbm_model = joblib.load(model_path)

# Sauvegarder le modèle (si nécessaire)
joblib.dump(best_lgbm_model, 'lightgbm_model.pkl')


['lightgbm_model.pkl']

In [1]:
import lightgbm as lgb

print("Version de LightGBM :", lgb.__version__)


Version de LightGBM : 4.5.0


In [10]:
import pandas as pd

# Chemin vers le fichier source et fichier nettoyé
source_file = "final_processed_data.csv"
cleaned_file = "final_processed_data_clean2.csv"

# Fonction pour charger les données ligne par ligne et enregistrer uniquement les lignes valides
def clean_csv_file(source_file, cleaned_file):
    with open(source_file, "rb") as f_in, open(cleaned_file, "w", encoding="utf-8") as f_out:
        line_num = 0
        for line in f_in:
            try:
                # Décode chaque ligne et écrit dans le nouveau fichier si la ligne est valide
                decoded_line = line.decode("utf-8")
                f_out.write(decoded_line)
            except UnicodeDecodeError:
                # En cas d'erreur d'encodage, signaler la ligne et ignorer
                print(f"Erreur d'encodage à la ligne {line_num}. Ligne ignorée.")
            line_num += 1

# Appeler la fonction pour créer un fichier nettoyé
clean_csv_file(source_file, cleaned_file)

# Charger les données nettoyées dans un DataFrame
reference_data = pd.read_csv(cleaned_file)


In [12]:
import pandas as pd
import numpy as np
import gc

# Charger les données d'entraînement
df = pd.read_csv('application_train.csv')
test_df = pd.read_csv('application_test.csv')
df = df._append(test_df).reset_index()

# Nettoyage des colonnes
df.columns = df.columns.str.replace('[^A-Za-z0-9]+', '', regex=True)
df.columns = df.columns.str.lower()

# Appliquer one-hot encoding aux colonnes catégorielles
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

df, new_columns = one_hot_encoder(df)

# Ajouter les nouvelles colonnes créées
df['daysemployed'].replace(365243, np.nan)
df['daysemployedperc'] = df['daysemployed'] / df['daysbirth']
df['incomecreditperc'] = df['amtincometotal'] / df['amtcredit']
df['incomeperperson'] = df['amtincometotal'] / df['cntfammembers']
df['annuityincomeperc'] = df['amtannuity'] / df['amtincometotal']
df['paymentrate'] = df['amtannuity'] / df['amtcredit']

# Afficher et sauvegarder les noms des colonnes après transformation
feature_names = df.columns
print(feature_names)

# Sauvegarder les noms des colonnes dans un fichier texte
with open('real_feature_names.txt', 'w') as f:
    for col in feature_names:
        f.write(col + '\n')

print("Les features ont été sauvegardées dans 'real_feature_names.txt'")


Index(['index', 'skidcurr', 'target', 'cntchildren', 'amtincometotal',
       'amtcredit', 'amtannuity', 'amtgoodsprice', 'regionpopulationrelative',
       'daysbirth',
       ...
       'wallsmaterialmode_Wooden', 'wallsmaterialmode_nan',
       'emergencystatemode_No', 'emergencystatemode_Yes',
       'emergencystatemode_nan', 'daysemployedperc', 'incomecreditperc',
       'incomeperperson', 'annuityincomeperc', 'paymentrate'],
      dtype='object', length=268)
Les features ont été sauvegardées dans 'real_feature_names.txt'


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.sklearn


In [ ]:
import pandas as pd

# Chargement des données
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')

# Fusion des deux fichiers pour avoir un ensemble unique
df = pd.concat([application_train, application_test], axis=0)

In [2]:
# Charger le fichier application_train.csv
df = pd.read_csv('application_train.csv')

# Supprimer les colonnes non numériques (si nécessaire)
df = df.select_dtypes(include=[np.number])

# Séparer les features et la target
X = df.drop(columns=['TARGET'])
y = df['TARGET']


In [3]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un modèle RandomForest pour l'importance des features
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Récupérer l'importance des features
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]  # Indices des features triées par importance

# Sélectionner les 50 meilleures features
top_50_indices = indices[:50]
X_train_top50 = X_train.iloc[:, top_50_indices]
X_test_top50 = X_test.iloc[:, top_50_indices]

# Obtenir les noms des features sélectionnées
selected_features = X_train.columns[top_50_indices]


In [4]:
# Entraîner un modèle RandomForest avec les 50 meilleures features
model_top50 = RandomForestClassifier(n_estimators=100, random_state=42)
model_top50.fit(X_train_top50, y_train)

# Prédire sur l'ensemble de test
y_pred = model_top50.predict_proba(X_test_top50)[:, 1]

# Calculer l'AUC
auc_score = roc_auc_score(y_test, y_pred)
print(f'AUC Score: {auc_score:.4f}')


AUC Score: 0.7093


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://127.0.0.1:5000/")
# Configurer l'URI de suivi
#mlflow.set_tracking_uri("file:///G:/projet data/OPC_Projets/P7/mlruns")

# Démarrer une nouvelle run sur MLflow
with mlflow.start_run():
    # Enregistrer le modèle
    mlflow.sklearn.log_model(model_top50, "model")

    # Enregistrer l'AUC
    mlflow.log_metric("AUC", auc_score)

    # Enregistrer les features sélectionnées
    mlflow.log_param("selected_features", list(selected_features))
    
    print("Modèle enregistré avec succès sur MLflow.")


2024/08/29 13:45:09 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/29 13:45:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run powerful-newt-338 at: http://127.0.0.1:5000//#/experiments/0/runs/c19188d0a61d47efb24a346686e3f98f.
2024/08/29 13:45:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000//#/experiments/0.


Modèle enregistré avec succès sur MLflow.


In [12]:
import requests

url = "http://127.0.0.1:5000/predict"
payload = {"SK_ID_CURR": 100002}
headers = {"Content-Type": "application/json"}

response = requests.post(url, json=payload, headers=headers)

# Afficher le code de statut HTTP et le contenu brut de la réponse
print("Status Code:", response.status_code)
print("Response Text:", response.text)

# Tenter de décoder en JSON si possible
try:
    print("JSON Response:", response.json())
except ValueError as e:
    print("Error decoding JSON:", e)


Status Code: 404
Response Text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
